# Use an existing Random Forest model to classify new BASE-9 data

This will likely be the main notebook for users (after creating the model in `create_model.ipynb`, if necessary).

This notebook performs the following tasks:
- reads in posterior data from BASE-9
- generates features from these data
- uses the model created in the previous notebook to classify the posterior sampling as "Good" vs. "Bad".  

In this example, we apply the model to data for NGC 6819, generated by Elizabeth Jeffery.

Most of the "heavy lifting" is done by the code in the `base9_ml_utils.py` file.  See the comments and markdown in that code for more details.

___
*Authors:* Justyce Watson, Aaron Geller\
*Date:* August 2025

## Import all functions from the `base9_ml_utils.py` file

In [1]:
# import functions from .py file
from base9_ml_utils import *

# The lines below are useful if you plan to make changes to the base9_ml_utils.py file.
# They will allow the notebook to refresh when you save changes to the .py file.
#
# %load_ext autoreload
# %autoreload 2


## Read in `.res` files and creates the features 

The user should specify the data directory on their own computer.  The code assumes that this directory contains one `.res` file for each star with the filename containing the star ID.  (If there is additional text in the file name, the user can specify this in the code, using the `file_prefix` and/or `file_suffix` args so that the code can identify the star ID from the filename properly.)  

We will use the `create_features` function imported from `base9_ml_utils.py`.

In [2]:
# run this cell to see information about this function
create_features?

Signature:
create_features(
    directory,
    column=0,
    max_nfiles=inf,
    file_prefix='NGC_2682_',
    file_suffix='',
    ess_num_samples=10000,
)
Docstring:
function that will calculate all the features needed for the ML model 
Note that the file names for the res files must contain the ids (and can include a prefix and suffix)

inputs:
- directory : (string) path to the data directory that contains the res files from BASE-9
- column : (string) column number to use from the res file to use to calculate features
- max_nfiles : (int) maximum number of files to use
- file_prefix : (string) prefix in the res file names before the id 
- file_suffix : (string) suffix in the res file names after the id
- ess_num_samples : (int) number of samples to use in ess normal distribution

outputs:
- pandas DataFrame with the calculated features (see code for more details)
File:      ~/Desktop/CIERA_files/BASE9_ML/BASE9_ML-1/base9_ml_utils.py
Type:      function

In [3]:
# directory on your computer where the .res data files are stored
directory = 'data/NGC6819/ngc6819_single_resfiles' 

# create a DataFrame with features for each star using the 'create_features'
ngc6819_statistic = create_features(directory, file_prefix='gaia_', file_suffix='_sin2')
 
# display the resulting DataFrame in the notebook
ngc6819_statistic


,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,2076279137552154112,1.611105,0.281932,0.241271,0.709164,12.619723,0.0,0.015125,0.158327,7.267119e-138,9825.036401
1,2076298933067389056,0.851659,0.281932,0.241271,0.492672,18.154950,0.0,0.075300,0.253846,0.000000e+00,9898.567992
2,2076584355102288128,1.568087,0.281932,0.241271,0.706918,12.674155,0.0,0.015795,0.156194,4.886454e-132,10141.620829
3,2076488525792609792,0.384405,0.281932,0.241271,0.494811,18.364221,0.0,0.052020,0.275837,0.000000e+00,10244.479209
4,2076581851147039232,1.684027,0.281932,0.241271,0.748072,11.787783,0.0,0.021270,0.173846,1.122364e-163,10264.491490
...,...,...,...,...,...,...,...,...,...,...,...
1683,2076393834663325568,1.420736,0.281932,0.241271,0.685752,12.768550,0.0,0.009732,0.160637,6.269840e-142,9841.208155
1684,2076380400005524224,1.370467,0.281932,0.241271,0.686048,12.883365,0.0,0.021400,0.142267,1.629872e-109,10078.826961
1685,2076299895139969536,0.444381,0.281932,0.241271,0.175729,54.419198,0.0,0.085369,0.166024,1.966690e-150,9501.128156
1686,2076393662864610688,1.519873,0.281932,0.241271,0.696125,12.398271,0.0,0.016644,0.163108,2.433374e-146,8967.509974


# Load saved model 

We assume that you have a model generated as in the `create_model.ipynb` notebook.  Here we saved the model as `my_model.pkl`.  Note that in order to use a saved model, you will need to be working with the same version of scipy (and possibly other dependencies).    

In [4]:
# Read in the saved model (as a scipy pipeline object)
pipe = load_model('my_model.pkl')

## Use the model to generate labels
Here we use the `make_preds` function imported from `base9_ml_utils.py`.  In this function we read in above and data to be labeled.  Here we will use the data from NGC 6819 that we created above.  Note that the `make_preds` function can accept either a pandas DataFrame (as we have here) or a numpy array.  If passed a DataFrame it will ensure that the columns are in the correct order and convert it to a numpy array.  If passed a numpy array, the user must ensure that the columns are already in the correct order.


*Note:* If you encounter an error here related to `scipy` it may mean that the model (read in above) was created using a different version of `scipy`.  There may be other dependency issues that I am not currently aware of as well.  If you see an error, you should remake the model on your machine using the same environment that you intend to run this notebook in.  If you need access to the data we used to create the model, please contact Aaron Geller.

In [5]:
# use the model to label the NGC 6819 data
y_pred_6819 = make_preds(pipe, ngc6819_statistic)

# print the outpt
y_pred_6819

array(['Bad', 'Bad', 'Bad', ..., 'Good', 'Bad', 'Good'], dtype=object)

In [6]:
# As a quick sanity check, you may want to print the number of labels that are "Good" vs "Bad"
# (You would expect a non-zero amount of each label type.)

print(
f'The model predicts {len(np.where(y_pred_6819 == "Good")[0])} "Good" \
and {len(np.where(y_pred_6819 == "Bad")[0])} "Bad" labels for these data.'
)

The model predicts 372 "Good" and 1316 "Bad" labels for these data.
